In [3]:
class GenerateArrowPlot(object):
    
    CHUNK_SVG = '<rect x="{}" y="0" width="{}" height="{}" style="stroke-width:1;stroke:{};fill:{}" />\n'
    LINE_SVG = '<line x1="{}" y1="{}" x2="{}" y2="{}" stroke="{}" stroke-width="1" />\n'
    ARROW_SVG = '<polygon points="{}" style="fill:{};stroke:{};stroke-width:1" />\n'
    
    def __init__(self, arrow_data, config_manager, arrow_width):
        self.arrow_data = arrow_data
        
        # Extract colors:
        arrow_colors = config_manager.get_arrow_colors()
        self.line_color = arrow_colors['line_color']
        self.utr_color = arrow_colors['utr_color']
        self.cds_color = arrow_colors['cds_color']
        
        # Extract other values
        self.chunk_size = config_manager.get_chunk_size()
        self.pixel_size = config_manager.get_pixel()
        
        self.arrow_width = arrow_width
        
    def generate_arrow_polt(self, gene_name):

        # Get CDS and UTR for a given gene:
        gene_df = (
            self.arrow_data
            .loc[lambda df: df.gene_name == gene_name]
            .sort_values('start')
            .reset_index(drop=True)
            .drop(['chr', 'gene_name', 'gene_id'], axis=1)
        )
        
        # If nothing found raise:
        assert len(gene_df) > 1, f'Gene name ({gene_name}) was not found in the data.'

        # Scaling genomic coordinates to screen coordinates:
        gene_df = (
            gene_df
            .assign(
                relative_start = lambda row: (row['start'] - gene_df.start.min())/self.chunk_size * self.pixel_size,
                relative_end = lambda row: (row['end'] - gene_df.start.min())/self.chunk_size * self.pixel_size,
            )
            .assign(
                length = lambda row: row['relative_end'] - row['relative_start']
            )
        )

        # Calculate intron midpoints:
        gene_df['next_start'] = (
            gene_df.relative_start.drop(0)
            .reset_index(drop=True)
        )
        
        gene_df['midpoint'] = (gene_df.next_start - gene_df.relative_end)/2 + gene_df.relative_end
        self.df = gene_df.head()
        
        # Generate boxes:
        boxes = gene_df.apply(self.draw_box, axis=1).dropna()
        
        # Generate lines connecting boxes:
        lines = gene_df.loc[gene_df.midpoint.notna()].apply(self.draw_lines, axis=1).dropna()

        # Generate closing arrow:
        arrow = self.draw_arrow(gene_df)
        
        # Concatenate into single string:
        svg_string = ''.join(lines.to_list() + boxes.to_list()) + arrow
        svg_object = svg_handler(svg_string, gene_df.relative_end.max(), arrow_width)
        
        return svg_object

    def draw_arrow(self, df):
        
        arrow_width = self.arrow_width
        
        # Get orientation:
        row = df.iloc[0] if df.iloc[0]['strand'] == '-' else df.tail(1).iloc[0]

        if row['strand'] == '+':
            coordinates = [
                (row['relative_start'],0), (row['relative_end'], 0), 
                (row['relative_end'], -arrow_width/2), (row['relative_end'] + arrow_width, arrow_width/2),
                (row['relative_end'], arrow_width * 1.5), (row['relative_end'],arrow_width), 
                (row['relative_start'], arrow_width)
            ]
        else:
            coordinates = [
                (row['relative_start'],0),(row['relative_end'], 0),
                (row['relative_end'], arrow_width), (row['relative_start'], arrow_width),
                (row['relative_start'], 1.5 * arrow_width), (row['relative_start'] - arrow_width, arrow_width / 2),
                (row['relative_start'], -arrow_width/2)
            ]

        coordinate_str = ' '.join([f'{x[0]},{x[1]}' for x in coordinates])

        return self.ARROW_SVG.format(coordinate_str, self.utr_color, self.line_color)
    
    def draw_box(self, row):

        if row['type'] == 'UTR':
            fill_color = self.utr_color
        elif row['type'] == 'CDS':
            fill_color = self.cds_color
        else:
            fill_color = 'black'

        return self.CHUNK_SVG.format(row['relative_start'], row['length'], self.arrow_width, self.line_color, fill_color)
    
    def draw_lines(self, row):
        
        if row['next_start'] - row['relative_end'] < 5:
            return None

        # Two bits are required:
        connector = self.LINE_SVG.format(row['relative_end'], 0, row['midpoint'], -self.arrow_width/2, self.line_color)
        connector += self.LINE_SVG.format(row['midpoint'], -self.arrow_width/2, row['next_start'], 0, self.line_color)

        return connector
        


svg_object =  gap.generate_arrow_polt('DYNLL1')
svg_object.group(translate=(10_000 / 450 * 18, 100))
print(svg_object.getSvg())

<g transform="translate(400.0 100)">
<line x1="10.16" y1="0" x2="536.3999999999999" y2="-25.0" stroke="#00CED1" stroke-width="1" />
<line x1="536.3999999999999" y1="-25.0" x2="1062.6399999999999" y2="0" stroke="#00CED1" stroke-width="1" />
<line x1="1068.1200000000001" y1="0" x2="1098.52" y2="-25.0" stroke="#00CED1" stroke-width="1" />
<line x1="1098.52" y1="-25.0" x2="1128.92" y2="0" stroke="#00CED1" stroke-width="1" />
<rect x="0.0" y="0" width="10.16" height="50" style="stroke-width:1;stroke:#00CED1;fill:#FF1493" />
<rect x="1062.6399999999999" y="0" width="0.20000000000004547" height="50" style="stroke-width:1;stroke:#00CED1;fill:#FF1493" />
<rect x="1062.88" y="0" width="5.240000000000009" height="50" style="stroke-width:1;stroke:#00CED1;fill:#FFD700" />
<rect x="1128.92" y="0" width="5.3599999999999" height="50" style="stroke-width:1;stroke:#00CED1;fill:#FFD700" />
<rect x="1134.3200000000002" y="0" width="11.399999999999864" height="50" style="stroke-width:1;stroke:#00CED1;fill:

In [7]:
import sys

import pandas as pd
import numpy as np
import logging

sys.path
sys.path.append('/Users/dsuveges/repositories/GenomePlotter')

from functions.CustomGenePlotter import CustomGeneIntegrator
from functions.ConfigManager import ConfigManager
from functions.svg_handler import svg_handler
from functions.ColorFunctions import ColorPicker
from functions.ChromosomePlotter import ChromosomePlotter

# Config manager:
cm = ConfigManager('/Users/dsuveges/repositories/GenomePlotter/config.updated.json')

# Color picker:
cp = ColorPicker(cm.get_chromosome_colors(), 0.9, 0.9, 20, 4000)

xf = pd.read_csv('/Users/dsuveges/repositories/GenomePlotter/source_data/processed_gencode_arrow.bed.gz', sep='\t', compression='infer')
arrow_width=50
gap = GenerateArrowPlot(xf, cm, arrow_width=arrow_width)

In [12]:
gene_name = 'PPP1R11'

# Get custom gene integrator:
cgi = CustomGeneIntegrator(gene_name, cm)
cgi.integrate(cp)

# Get chromosome plotter:
chrp = ChromosomePlotter(cgi.get_integrated_data(), 18)

chrp.draw_chromosome()
gene_svg = chrp.return_svg()

# Get arrow:
gap = GenerateArrowPlot(xf, cm, arrow_width=arrow_width)
svg_object =  gap.generate_arrow_polt(gene_name)
svg_object.group(translate=(10_000 / 450 * 18, 100))

gene_svg += svg_object.getSvg()

print(gene_svg)

<rect x="3492" y="0" width="18" height="18" style="stroke-width:1;stroke:#c4ebe1; fill: #c4ebe1" />
<rect x="3510" y="0" width="18" height="18" style="stroke-width:1;stroke:#c4ebe1; fill: #c4ebe1" />
<rect x="3528" y="0" width="18" height="18" style="stroke-width:1;stroke:#c9ede4; fill: #c9ede4" />
<rect x="3546" y="0" width="18" height="18" style="stroke-width:1;stroke:#c4ebe1; fill: #c4ebe1" />
<rect x="3564" y="0" width="18" height="18" style="stroke-width:1;stroke:#c0e9df; fill: #c0e9df" />
<rect x="3582" y="0" width="18" height="18" style="stroke-width:1;stroke:#c0e9df; fill: #c0e9df" />
<rect x="0" y="18" width="18" height="18" style="stroke-width:1;stroke:#c4ebe1; fill: #c4ebe1" />
<rect x="18" y="18" width="18" height="18" style="stroke-width:1;stroke:#ceeee6; fill: #ceeee6" />
<rect x="36" y="18" width="18" height="18" style="stroke-width:1;stroke:#ceeee6; fill: #ceeee6" />
<rect x="54" y="18" width="18" height="18" style="stroke-width:1;stroke:#d3f0e9; fill: #d3f0e9" />
<rect

In [ ]:
from functions.ChromosomePlotter import ChromosomePlotter

cp = ChromosomePlotter(cgi.get_integrated_data(), 18)

In [ ]:
cp.draw_chromosome()

In [ ]:
print(cp.return_svg())